In [2]:
# !pip install -e ".[icechunk]"
# !pip install git+https://github.com/mpiannucci/kerchunk@v3
# !pip install fsspec s3fs

In [3]:
import warnings

import fsspec
import xarray as xr
from icechunk import IcechunkStore, StorageConfig, StoreConfig, VirtualRefConfig

from virtualizarr import open_virtual_dataset

warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
fs = fsspec.filesystem("s3", anon=True)

oisst_files = fs.glob(
    "s3://noaa-cdr-sea-surface-temp-optimum-interpolation-pds/data/v2.1/avhrr/202408/oisst-avhrr-v02r01.*.nc"
)

oisst_files = sorted(["s3://" + f for f in oisst_files])

In [5]:
so = dict(anon=True, default_fill_cache=False, default_cache_type="none")

virtual_datasets = [
    open_virtual_dataset(url, indexes={}, reader_options={"storage_options": so})
    for url in oisst_files[0:2]
]

In [6]:
virtual_ds = xr.concat(
    virtual_datasets,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

In [8]:
#!rm -rf ./noaa-cdr-icechunk/

In [7]:
storage_config = StorageConfig.filesystem("./noaa-cdr-icechunk")
virtual_ref_store_config = StoreConfig(
    virtual_ref_config=VirtualRefConfig.s3_anonymous(region="us-east-1"),
)

In [8]:
store = IcechunkStore.create(storage=storage_config, config=virtual_ref_store_config)

In [9]:
virtual_ds.virtualize.to_icechunk(store)

In [10]:
store.commit("first 2 days of 202408 data")

'MZWAPPV1HD75JFQFCZ2G'

In [11]:
ds = xr.open_zarr(store, consolidated=False, zarr_format=3)
ds

<xarray.Dataset> Size: 66MB
Dimensions:  (lon: 1440, time: 2, zlev: 1, lat: 720)
Coordinates:
  * lon      (lon) float32 6kB 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * time     (time) datetime64[ns] 16B 2024-08-01T12:00:00 2024-08-02T12:00:00
  * lat      (lat) float32 3kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * zlev     (zlev) float32 4B 0.0
Data variables:
    err      (time, zlev, lat, lon) float64 17MB ...
    sst      (time, zlev, lat, lon) float64 17MB ...
    anom     (time, zlev, lat, lon) float64 17MB ...
    ice      (time, zlev, lat, lon) float64 17MB ...
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    cdm_data_type:              Grid
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    creator_email:              oisst-help@noaa.gov
    creator_url:                https://www.ncei.noaa.gov/
    date_created:               2024-08-16T09:12:00Z
    ...                         ...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    standard_name_vocabulary:   CF Standard Name Table (v40, 25 January 2017)
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    time_coverage_end:          2024-08-01T23:59:59Z
    time_coverage_start:        2024-08-01T00:00:00Z
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...

## Append

In [12]:
virtual_datasets_a = [
    open_virtual_dataset(
        url, indexes={}, reader_options={"storage_options": {"anon": True}}
    )
    for url in oisst_files[2:4]
]

In [13]:
virtual_ds_a = xr.concat(
    virtual_datasets_a,
    dim="time",
    coords="minimal",
    compat="override",
    combine_attrs="override",
)

In [14]:
virtual_ds_a

<xarray.Dataset> Size: 17MB
Dimensions:  (time: 2, zlev: 1, lat: 720, lon: 1440)
Coordinates:
    zlev     (zlev) float32 4B ManifestArray<shape=(1,), dtype=float32, chunk...
    lat      (lat) float32 3kB ManifestArray<shape=(720,), dtype=float32, chu...
    time     (time) float32 8B ManifestArray<shape=(2,), dtype=float32, chunk...
    lon      (lon) float32 6kB ManifestArray<shape=(1440,), dtype=float32, ch...
Data variables:
    err      (time, zlev, lat, lon) int16 4MB ManifestArray<shape=(2, 1, 720,...
    ice      (time, zlev, lat, lon) int16 4MB ManifestArray<shape=(2, 1, 720,...
    sst      (time, zlev, lat, lon) int16 4MB ManifestArray<shape=(2, 1, 720,...
    anom     (time, zlev, lat, lon) int16 4MB ManifestArray<shape=(2, 1, 720,...
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.20240803.nc
    naming_authority:           gov.noaa.ncei
    ...                         ...
    time_coverage_start:        2024-08-03T00:00:00Z
    time_coverage_end:          2024-08-03T23:59:59Z
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR

In [15]:
append_store = IcechunkStore.open_existing(
    storage=storage_config, config=virtual_ref_store_config, mode="a"
)

In [16]:
await append_store._store.get("time/c/0")

b'x^cx\xd3\xe2\x06\x00\x04\x16\x01\xb7'

In [17]:
virtual_ds_a.virtualize.to_icechunk(append_store, append_dim="time")

In [18]:
append_store.commit("wrote 2 more days of data")

'JZXQ42REXTMJGCYZ2GRG'

In [19]:
read_store = IcechunkStore.open_existing(
    storage=storage_config, config=virtual_ref_store_config, mode="r"
)

In [20]:
await read_store._store.get("time/c/0")

b'\x00\x00\x00>\x00\x00\xc0>\x00\x00 ?\x00\x00`?\x00\x00\x90?\x00\x00\xb0?\x00\x00\xd0?\x00\x00\xf0?\x00\x00\x08@\x00\x00\x18@\x00\x00(@\x00\x008@\x00\x00H@\x00\x00X@\x00\x00h@\x00\x00x@\x00\x00\x84@\x00\x00\x8c@\x00\x00\x94@\x00\x00\x9c@\x00\x00\xa4@\x00\x00\xac@\x00\x00\xb4@\x00\x00\xbc@\x00\x00\xc4@\x00\x00\xcc@\x00\x00\xd4@\x00\x00\xdc@\x00\x00\xe4@\x00\x00\xec@\x00\x00\xf4@\x00\x00\xfc@\x00\x00\x02A\x00\x00\x06A\x00\x00\nA\x00\x00\x0eA\x00\x00\x12A\x00\x00\x16A\x00\x00\x1aA\x00\x00\x1eA\x00\x00"A\x00\x00&A\x00\x00*A\x00\x00.A\x00\x002A\x00\x006A\x00\x00:A\x00\x00>A\x00\x00BA\x00\x00FA\x00\x00JA\x00\x00NA\x00\x00RA\x00\x00VA\x00\x00ZA\x00\x00^A\x00\x00bA\x00\x00fA\x00\x00jA\x00\x00nA\x00\x00rA\x00\x00vA\x00\x00zA\x00\x00~A\x00\x00\x81A\x00\x00\x83A\x00\x00\x85A\x00\x00\x87A\x00\x00\x89A\x00\x00\x8bA\x00\x00\x8dA\x00\x00\x8fA\x00\x00\x91A\x00\x00\x93A\x00\x00\x95A\x00\x00\x97A\x00\x00\x99A\x00\x00\x9bA\x00\x00\x9dA\x00\x00\x9fA\x00\x00\xa1A\x00\x00\xa3A\x00\x00\xa5A\x00\x00\xa7A\x00

In [21]:
ds = xr.open_zarr(read_store, consolidated=False, zarr_format=3)
ds

error: Failed to decode variable 'time': Error -3 while decompressing data: unknown compression method